В самостоятельном проекте вы будете работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.

Вы будете работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах вы найдёте информацию и о более поздних оценках, которые эти посты получили. 

# Часть 1

1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(p.id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id
WHERE type = 'Question' AND (score > 300 OR favorites_count >= 100)

2.Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
WITH cnt AS
(      SELECT creation_date::date,
       COUNT(id) OVER (PARTITION BY creation_date::date) AS cnt
FROM stackoverflow.posts
WHERE (creation_date::date BETWEEN '2008-11-01' AND '2008-11-18') AND post_type_id = 1
)
SELECT 383

3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
JOIN stackoverflow.badges AS b on u.id=b.user_id
WHERE u.creation_date::date = b.creation_date::date

4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
JOIN stackoverflow.votes v ON p.id = v.post_id
WHERE u.display_name = 'Joel Coehoorn'

5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
ROW_NUMBER() OVER(ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;

6. Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT DISTINCT u.id,
       COUNT(v.vote_type_id) OVER (PARTITION BY u.id) AS total_votes
FROM stackoverflow.users u 
JOIN stackoverflow.votes v on u.id=v.user_id
WHERE v.vote_type_id = 6
ORDER BY total_votes DESC,
         u.id DESC
LIMIT 10;

7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT u.id, 
       COUNT(b.id), 
       DENSE_RANK() OVER (ORDER BY COUNT(b.id) DESC) as rating
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE b.creation_date::date between '2008-11-15' AND '2008-12-15'
GROUP BY u.id
ORDER BY COUNT(b.id) DESC, u.id 
LIMIT 10;

8. Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT p.title,
       u.id,
       p.score,
       ROUND(AVG(score) OVER (PARTITION BY u.id))
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON p.user_id=u.id
WHERE title is not null and score != 0;

9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
WITH bd AS 
(SELECT DISTINCT 
        u.id AS id,
        COUNT(b.id) AS sum_bd
FROM stackoverflow.users AS u 
INNER JOIN stackoverflow.badges AS b ON b.user_id=u.id
GROUP BY u.id
HAVING COUNT(b.id)>1000)

SELECT p.title
FROM stackoverflow.posts AS p 
INNER JOIN bd ON bd.id=p.user_id
WHERE p.title !='';

10. Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). 

Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
       views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views >= 100 THEN 2
           ELSE 3
       END as group
FROM stackoverflow.users
WHERE location LIKE '%United States%'
 and views != 0

11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH df AS (SELECT id,
views,
CASE 
    WHEN views >= 350 THEN 1
    WHEN views <= 350 AND views >= 100 THEN 2
    WHEN views <= 100 THEN 3
END AS group_name
FROM stackoverflow.users
WHERE views <> 0
AND location LIKE '%United States%'),
 
df_2 AS (SELECT id,
group_name,        
MAX(views) OVER(PARTITION BY group_name ORDER BY views) AS top
FROM df),
 
df_3 AS (SELECT *,
RANK() OVER(PARTITION BY group_name ORDER BY top DESC) AS top_place
FROM df_2)
 
SELECT id,
group_name, 
top
FROM df_3
WHERE top_place = 1
ORDER BY 3 DESC, 1

12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
WITH a AS (
SELECT distinct(EXTRACT(DAY FROM u.creation_date::DATE)) AS DAY,
       count(u.id) AS b
FROM stackoverflow.users u
WHERE u.creation_date::DATE BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY 1
)
SELECT DAY, 
       b,
       SUM(b) OVER (ORDER BY DAY) AS costs_cum
       
FROM a

13.Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. 

Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
WITH t1 AS
(select u.id AS user_id,
       u.creation_date AS user_creation_date,
       p.creation_date AS post_creation_date,
       ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY p.creation_date)
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON p.user_id=u.id)

SELECT user_id,
       post_creation_date - user_creation_date
FROM t1
WHERE row_number = 1

# Часть 2

1. Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date,
       SUM(views_count)
FROM stackoverflow.posts
GROUP BY 1
ORDER BY 2 DESC

2. Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
with t as (
    select
    display_name,
    count(distinct user_id) as user_ids,  
    count(p.id) as post_count

from stackoverflow.users u
join stackoverflow.posts p on u.id = p.user_id
join stackoverflow.post_types pt on p.post_type_id = pt.id
where type = 'Answer' and DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'
group by display_name
)

select 
    display_name,
    user_ids
from t
group by display_name, user_ids, post_count
having post_count > 100
order by display_name

3. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH
i AS (SELECT u.id
      FROM stackoverflow.users AS u
      WHERE DATE_TRUNC('month', u.creation_date) BETWEEN '2008-09-01'
      AND '2008-09-30'),
b AS (SELECT p.user_id,
             COUNT(p.id) AS cnt_posts
      FROM stackoverflow.posts AS p
      WHERE DATE_TRUNC('month', p.creation_date) BETWEEN '2008-12-01'
      AND '2008-12-31'
      GROUP BY 1
      HAVING COUNT(p.id) >= 1)
SELECT DATE_TRUNC('month', p.creation_date) :: date AS month,
       COUNT(p.id) AS cnt_posts
FROM i
JOIN b ON i.id = b.user_id
JOIN stackoverflow.posts AS p ON i.id = p.user_id
GROUP BY month
ORDER BY month DESC;

4.Используя данные о постах, выведите несколько полей:

- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id, 
       creation_date,
       views_count, 
       sum(views_count) over (partition by user_id order by creation_date asc ) as views_cum 
FROM stackoverflow.posts 
ORDER by 1,2;

5. Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
WITH a AS
(SELECT user_id,
       creation_date,
       COUNT(user_id) OVER (PARTITION BY user_id) as cnt
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY 1, 2)
SELECT ROUND(AVG(cnt))
FROM a

6. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
SELECT month,
       current_cnt_posts,
       ROUND((current_cnt_posts :: numeric / previous_cnt_posts - 1) * 100, 2) AS change_previous_month
FROM (
    SELECT *,
           LAG(current_cnt_posts) OVER(ORDER BY month) AS previous_cnt_posts
    FROM (
        SELECT EXTRACT(MONTH FROM p.creation_date) AS month,
               COUNT(p.id) AS current_cnt_posts
        FROM stackoverflow.posts AS p
        WHERE DATE_TRUNC('month', p.creation_date) BETWEEN '2008-09-01'
        AND '2008-12-31'
        GROUP BY month
        ORDER BY month) AS i) AS a;

7. Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
SELECT DISTINCT EXTRACT(WEEK FROM p.creation_date) AS week_number,
       LAST_VALUE(p.creation_date) OVER(PARTITION BY EXTRACT(WEEK FROM p.creation_date) ORDER BY p.creation_date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_dt_post 
FROM (
    SELECT DISTINCT FIRST_VALUE(user_id) OVER() AS first_user
    FROM (
        SELECT p.user_id AS user_id,
               COUNT(p.id) AS count_posts
        FROM stackoverflow.posts AS p
        GROUP BY user_id
        ORDER BY count_posts DESC) AS i) AS b
JOIN stackoverflow.posts AS p ON b.first_user = p.user_id
WHERE DATE_TRUNC('month', p.creation_date) BETWEEN '2008-10-01' 
AND '2008-10-31';